In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm
import re

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

### 데이터 확인 - clinical 상위 체계

In [ ]:
data = pd.read_csv("./drive/MyDrive/DW_metformin/DQM A Concept Sets.csv") 
cdc= data[data['Class']=='Clinical Drug Comp']
cdg= data[data['Class']=='Clinical Dose Group'] #병용 성분 표기 
cdf= data[data['Class']=='Clinical Drug Form']  #병용 성분 표기 

In [ ]:
#CDG - 성분만 추출, 용량은 없음. 
import re
cdg_name =cdg['Name']
cdg_ingredient = [re.sub("metformin|Oral|Product|Pill|/|Liquid", "",i) for i in cdg_name] #특정 문자 제거 
cdg_ingredient= [i.strip() for i in cdg_ingredient] #공백제거 
cdg_ingredient = list(set(cdg_ingredient))
print(cdg_ingredient)

In [ ]:
# CDF-성분만 추출 
cdf_name = cdf['Name']
cdf_ingredient = [re.sub("Powder|Solution|for|metformin|Metformin|Oral|Tablet|Suspension|Injectable Solution|Capsule|Extended Release|/|Delayed Release", "",i) for i in cdf_name] #특정 문자 제거 
cdf_ingredient= [i.strip() for i in cdf_ingredient] #공백제거 
cdf_ingredient = list(set(cdf_ingredient))
cdf_ingredient

### 데이터 확인 - Branded-하위체계
#### Branded Drug Comp --> 성분명,용량?`
#### Branded Drug Form --> 성분명

In [ ]:
# Branded Drug Comp 
# 복합제 성분만 우선 추출: / 가 있는 것만 추출 
# 숫자, MG, 상품명 제거 []안에 든 문자 모두 제거 
# metformin 제거  
bdc= data[data['Class']=='Branded Drug Comp']
bdc.head()

### 병용 약물 확인

In [ ]:
## 병용 성분 확인 (branded drug comp) 
# branded drug comp 우선, labeling 
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+\s?[a-zA-Z]+) [0-9]+? MG? /? (?P<ingredient2>[a-zA-Z]+\s?[a-zA-Z]+)? [0-9]+? MG? [[a-zA-Z]+]?')
matchobj= [ingredient.search(i)  for i in bdc["Name"]] # 복합제 및 병용만 추출  
ing=list(filter(None, matchobj))  # none값 필터링


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Possible nested set at position 107
  after removing the cwd from sys.path.


In [ ]:
## 병용 성분 확인 
ingredient1= [i.group(1) for i in ing]
ingredient1=list(set(ingredient1)) #중복제거 
ingredient2= [i.group(2) for i in ing]
ingredient2=list(set(ingredient2)) #중복제거 
total = ingredient1+ ingredient2
total = list(set(total))
total

' gnd: repaglinide, nateglinide\n    sglt-2: -ozin\n    statin: -statin\n    su: gl-\n    dpp4i: -liptin\n    tzd: -zone\n'

## 조건식으로 분류 
### 병용 성분 확인: 
#####clinical drug form (성분/성분/제형, 3제요법도 포함), 
#####clinical dose group (성분/성분 제형)
#####branded drug comp (성분 용량mg / 성분 용량mg [상품명])
#####branded drug form(성분/ 성분 제형 [상품명])

    gnd: -glinide
    sglt-2: -ozin
    alpha : -bose
    su: gl- or -amide
    dpp4i: -liptin
    tzd: -zone

    ** 제거  statin: -statin, ropinirole (치매약)


In [ ]:
# 병용 약물군 분류 
## 성분/성분 형식만: clinical drug form, clinical dose group, branded drug  form 
condition= (data.Class=='Clinical Drug Form') | (data.Class=='Branded Drug Form') |(data.Class=='Clinical Dose Group')
db1= data[condition]
db1.head()
## 병용 성분 갯수 세기: 3제 요법이 있는지 확인. 
db1['ingredient_count'] =   [i.count(' / ')+1 for i in db1['Name']]
##2제 요법만 골라서 분류 
db2= db1[db1.ingredient_count==2]

In [50]:
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+) / (?P<ingredeint2>[a-zA-Z]+)')
matchobj= [ingredient.search(i)  for i in db2["Name"]]  
matchobj
lists=[]
for v in matchobj:
  if  v.group(1).lower().startswith('gl') or v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide') or v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin') or v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin') or v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone') or v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide') or v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().endswith('bose') or v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db2['type']=lists
db2.head()

,Unnamed: 0,Unnamed: 0.1,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2,type


In [51]:
db2.to_csv('./test.csv')

In [52]:
# 1제 요법은 type= single
# 3제 요법은 type = error 
db11=db1[db1.ingredient_count ==1]
db13=db1[db1.ingredient_count ==3]
db11['type'] = 'single'
db13['type'] = 'error'


In [53]:
#1,2,3 제 요법 합치기. 
db = pd.concat([db11, db2, db13], axis=0)
db.head()

,Unnamed: 0,Unnamed: 0.1,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2,type
0,0,NaN,46234112,1653600,liraglutide Pen Injector [Saxenda],Branded Drug Form,Standard,0,"7,494",Drug,RxNorm,1,1,error,,single
1,1,NaN,46234111,1653597,liraglutide Pen Injector [Victoza],Branded Drug Form,Standard,0,0,Drug,RxNorm,1,1,error,,single
50,50,NaN,44121622,OMOP1116253,liraglutide Injectable Solution [Victoza],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,error,,single
51,51,NaN,44121403,OMOP1116034,pioglitazone Oral Tablet [Accel Pioglitazone],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,tzd,,single
52,52,NaN,44121402,OMOP1116033,pioglitazone Oral Tablet [Apo-Pioglitazone],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,tzd,,single


In [54]:
# branded drug comp (성분 용량mg / 성분 용량mg [상품명]) 분류 
## ingredient_count 병용약물 갯수 세기 
db1= data[data.Class=='Branded Drug Comp']
db1['ingredient_count'] =   [i.count(' / ')+1 for i in db1['Name']]
##2제 요법만 골라서 분류 
db2= db1[db1.ingredient_count==2]
db2.head()

NameError: ignored

In [ ]:
db2.head(7)

In [ ]:
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG / (?P<ingredeint2>[a-zA-Z]+[ ]?[a-zA-Z]+?[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG')
matchobj= [ingredient.search(i)  for i in db2["Name"]]  
matchobj

lists=[]
for v in matchobj:
  if v == None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl') or v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide') or v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin') or v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin') or v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone') or v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide') or v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide') or v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose') or v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db2['type']=lists
db2.head()

In [ ]:
db2.to_csv('test.csv')

In [ ]:
# 1제 요법은 type= single
# 3제 요법은 type = error 
db11=db1[db1.ingredient_count ==1]
db13=db1[db1.ingredient_count ==3]
db11['type'] = 'single'
db13['type'] = 'error'


In [ ]:
#1,2,3 제 요법 합치기. 
dbdb = pd.concat([db11, db2, db13], axis=0)
dbdb.head()

In [ ]:
# 합치기 
total = pd.concat([dbdb, db], axis=0)
total.head()

In [ ]:
db.head()

In [ ]:
dbdb.head()

In [ ]:
# 필요한 부분만 추출
adm_type =total[['Id', 'Code', 'Name', 'Class', 'type','ingredient_count']]

In [ ]:
adm_type.head()

In [ ]:
adm_type.to_csv("./drive/MyDrive/DW_metformin/adm_type.csv")

## 대조군 약물
###### 훨씬 많다.

""" 
---
제외 --RC/DRC 0 
Quant Clinical Box
Clinical Pack Box
Branded Pack Box
Quant Branded Box
Branded Drug Box
Branded Pack
Branded Dose Group--DRC만 많음, 상품명만

연결 1만 이하 
Quant Branded Drug
Quant Clinical Drug -숫자 글자2 성분 숫자 글자2/글자2 
---
연결 1만 이상
Ingredient    --간단하게 성분만 

Branded Drug : 성분 숫자 단위(MG) / 성분 숫자 단위 (RC/DRC모두 많다)
Clinical Drug --DRC만 많음, 이하동문
Branded Drug Comp--DRC만 많음, 이하동문
Marketed Product --성분 용량 MG / 성분 용량 MG, 주사제의 경우 다름. 용량 ML 성분 숫자 UNT/ML / 성분 숫자.MG/ML


Branded Drug Form--DRC만 많음, 성분/ 성분
Clinical Dose Group--DRC만 많음, 성분/성분
---
GLP-1 : 실험군에선 3제 병용으로 제외. 

"""


In [3]:
#대조군 데이터 확인 
# c1= pd.read_csv("./drive/MyDrive/DW_metformin/c1.csv")
# c2= pd.read_csv("./drive/MyDrive/DW_metformin/c2.csv")
# c3= pd.read_csv("./drive/MyDrive/DW_metformin/c3.csv")
# c4= pd.read_csv("./drive/MyDrive/DW_metformin/c4.csv")
# c= pd.concat([c1, c2, c3,c4], axis=0)
# c.to_csv('./drive/MyDrive/DW_metformin/c.csv')
# c.head()
c= pd.read_csv('./drive/MyDrive/DW_metformin/c.csv')

###### ingredient 분류 

In [ ]:
# ingredient 분류 
ingredient =c[c.Class=='Ingredient']
ingredient.head()

In [ ]:
name = ingredient['Name']
lists=[]
for v in name:
  if  v.lower().startswith('gl') or v.lower().startswith('gl'):
    lists.append('SU')
  elif v.lower().endswith('amide') or v.lower().endswith('amide'):
    lists.append('SU')
  elif v.lower().endswith('ozin') or v.lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.lower().endswith('liptin') or v.lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.lower().endswith('zone') or v.lower().endswith('zone'):
    lists.append('tzd')
  elif v.lower().endswith('glinide') or v.lower().endswith('glinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().startswith('mitiglinide') or v.lower().startswith('mitiglinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().endswith('bose') or v.lower().endswith('bose'):
    lists.append('alpha')
  elif v.lower().endswith('miglitol') or v.lower().endswith('miglitol'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
ingredient['type1']=lists
ingredient['type2']=''
ingredient['ingredient_count']= 1
ingredient.head()
#glp-1은 3제 요법으로, 보험 인정, 본 연구는 2제 요법을 대상으로 하므로 제외. 

In [34]:
ingredient.to_csv('./test.csv')

#### Branded/ Clinical Drug, Branded Drug Comp, Marketed Product
#### 형식 동일해 성분 용량 단위 형식 분류 
---
Branded Drug : 성분 숫자 단위(MG) / 성분 숫자 단위 (RC/DRC모두 많다)
Clinical Drug --DRC만 많음, 이하동문
Branded Drug Comp--DRC만 많음, 이하동문
Marketed Product --성분 용량 MG / 성분 용량 MG, 주사제의 경우 다름. 용량 ML 성분 숫자 UNT/ML / 성분 숫자.MG/ML

In [ ]:
# 병용 약물군 분류 
## 성분/성분 형식만: clinical drug form, clinical dose group, branded drug  form 
condition= (c.Class=='Branded Drug') | (c.Class=='Clinical Drug') |(c.Class=='Branded Drug Comp') | (c.Class=='Marketed Product')
db = c[condition]
db.head()
## 병용 성분 갯수 세기: 3제 요법이 있는지 확인. 
db['ingredient_count'] =   [i.count(' / ')+1 for i in db['Name']]
db.head()

In [ ]:
# 2제 요법만 골라서 분류. 
# 3제 요법은-error 표시 
db1 = db[db['ingredient_count'] ==1 ]
db2 = db[db['ingredient_count']==2]
db3 = db[db['ingredient_count']==3]
db3['type1'] = 'error'
db3['type2']='error'
db3.head()

In [ ]:
#1,2제 요법 나눠서 분류 
##1제 요법 
reg1= re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG ')
matchobj= [reg1.search(i)  for i in db1["Name"]]
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db1['type1']=lists
db1['type2']=''
db1.head()

In [ ]:
#2제 요법   
reg2 = re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG / (?P<ingredeint2>[a-zA-Z]+[ ]?[a-zA-Z]+?[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG')
matchobj= [reg2.search(i)  for i in db2["Name"]]  
matchobj

##type1
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
db2['type1']=lists

## type2
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
db2['type2']=lists

db2.head()

In [ ]:
# 합치기 
control_1= pd.concat([ingredient, db1, db2,db3], axis=0)
control_1.head()

### Branded Drug Form, Clinical Dose Group
#### DRC만 많음, 성분/ 성분


In [ ]:
condition= (c.Class=='Branded Drug Form') | (c.Class=='Clinical Dose Group')
db = c[condition]
db.head()

In [ ]:
#병용 약물 갯수 세기 
## 병용 성분 갯수 세기: 3제 요법이 있는지 확인. 
db['ingredient_count'] =   [i.count(' / ')+1 for i in db['Name']]
db.head()

In [ ]:
#3제 요법 확인 --type 1, type2 모두 error
db3= db[db['ingredient_count']==3]
db3['type1']='error'
db3['type2']='error'
db3.head()


In [ ]:
#1제 요법 확인 -type1, type2 =''
db1 = db[db['ingredient_count']==1]
#1,2제 요법 나눠서 분류 
##1제 요법 
reg1= re.compile('(?P<ingredient1>[a-zA-Z]+)')
matchobj= [reg1.search(i)  for i in db1["Name"]]
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db1['type1']=lists
db1['type2']=''
db1.head()

In [ ]:
#2제 요법   
reg2 = re.compile('(?P<ingredient1>[a-zA-Z]+) / (?P<ingredeint2>[a-zA-Z]+)')
matchobj= [reg2.search(i)  for i in db2["Name"]]  
matchobj

##type1
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
db2['type1']=lists

## type2
lists=[]
for v in matchobj:
  if v== None:
    lists.append('error')
  elif  v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
db2['type2']=lists

db2.head()

In [ ]:
# 합치기 
control= pd.concat([control_1, db1, db2,db3], axis=0)
control.head()

In [47]:
control.to_csv('./drive/MyDrive/DW_metformin/adm_type_control.csv')